# ![Specutils: An Astropy Package for Spectroscopy](data/specutils_logo.png)


This notebook provides an overview of the Specutils Astropy coordinated package.  While this notebook is intended as an interactive introduction to specutils at the time of its writing, the canonical source of information for the package is the latest version's documentation: 

https://specutils.readthedocs.io

# Installation

 

<div class="alert alert-block alert-warning">
If you followed the workshop instructions, specutils should be already installed. But depending on when you installed it, you might need to follow <a href="../../edit/00-Install_and_Setup/UPDATING.md">the UPDATING.md instructions</a> at the base of this repo.
</div>


If it is not already installed, installing `specutils` should be as straightforward as either:
```
$ conda install -c conda-forge specutils
```

or

```
$ pip install specutils
```

although for futher information see [the installation section of the docs](https://specutils.readthedocs.io/en/latest/installation.html).

Once these are installed, fundamental imports necessary for this notebook are possible:

In [ ]:
import numpy as np

import astropy.units as u

import specutils
from specutils import Spectrum1D
specutils.__version__

In [ ]:
# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt


# for showing quantity units on axes automatically:
from astropy.visualization import quantity_support
quantity_support();

# Background/Spectroscopic ecosystem

The large-scale plan for spectroscopy support in the Astropy project is outlined in  the [Astropy Proposal For Enhancement 13](https://github.com/astropy/astropy-APEs/blob/main/APE13.rst).  In summary, this APE13 lays out three broad packages:

* `specutils` - a Python package containing the basic data structures for representing spectroscopic data sets, as well as a suite of fundamental spectroscopic analysis tools to work with these data structures.
* `specreduce` - a general Python package to reduce raw astronomical spectral images to 1d spectra (represented as `specutils` objects).
* `specviz` - a Python package (or possibly suite of packages) for visualization of astronomical spectra.


While all are still in development, the first of these is furthest along, and is the subject of this notebook, as it contains the core data structures and concepts required for the others.

# Fundamentals of specutils

## Objects for representing spectra

The most fundamental purpose of `specutils` is to contain the shared Python-level data structures for storing astronomical spectra.  It is important to recognize that this is not the same as the *on-disk* representation.  As desecribed later specutils provides loaders and writers for various on-disk representations, with the intent that they all load to a common set of in-memory/Python interfaces.  Those intefaces (implemented as Python classes) are described in detail in the [relevant section of the documentation](https://specutils.readthedocs.io/en/latest/types_of_spectra.html), which contains this diagram:

![Specutils Classes](data/specutils_classes_diagrams.png)

The core principal is that all of these representations contain a `spectral_axis` attribute as well as a `flux` attribute (as well as optional matching `uncertainty`).  The former is often wavelength for OIR spectra, but might be frequency or energy for e.g. Radio or X-ray spectra.  Regardless of which spectral axis is used, the class attempts to interpret it appropriately, using the features of `astropy.Quantity` to distinguish different types of axes.  Similarly, `flux` may or may not be a traditional astronomical `flux` unit (e.g. Jy or  erg sec$^{-1}$ cm$^{-2}$ angstrom$^{-1}$), but is treated as the portion of the spectrum that acts in that manner.  The various classes are then distinguished by whether these attributes are one-dimensional or not, and how to map the `spectral_axis` dimensionality onto the `flux`.  The simplest case (and the one primarily considered here) is the scalar `Spectrum1D` case, which is a single spectrum with a matched-size `flux` and `spectral_axis`.

## Basics of creating Spectrum1D Objects

While often a spectrum will be loaded from a file (see below), if the format is not compatible, or particular customization is required, spectra can be greated directly from arrays and astropy quantities.  E.g.:

In [ ]:
flux = (np.random.randn(100)*0.2 + 2) * u.erg / u.s / (u.cm**2) / u.AA 
wavelength = np.linspace(3000, 8000, 100)*u.AA
spec1d = Spectrum1D(spectral_axis=wavelength, flux=flux)

plt.step(spec1d.spectral_axis, spec1d.flux)

spec1d

These can be straightforwardly transformed to other spectral units if desired:

In [ ]:
jyspec1d = spec1d.new_flux_unit(u.Jy)
plt.plot(jyspec1d.frequency, jyspec1d.flux)

## Working with Spectral Axes 

We created `Spectrum1D` just as Quantity arrays, so they can be treated just as `Quantity` objects when convenient with unit conversions and the like:

In [ ]:
spec1d.spectral_axis

In [ ]:
spec1d.spectral_axis.to(u.nm)

In [ ]:
spec1d.spectral_axis.to(u.THz, u.spectral())

But under the hood this are are fully-featured WCS following the [Astropy APE14](https://github.com/astropy/astropy-APEs/blob/main/APE14.rst) WCS interface along with the [GWCS](https://gwcs.readthedocs.io/) package. So you can use that to do conversions to and from spectral to pixel axes:

In [ ]:
spec1d.wcs.pixel_to_world([10, 10.5])

In [ ]:
spec1d.wcs.world_to_pixel([400, 450]*u.nm)

## Uncertainties

Currently the most compatible way to use uncertainties is the machinery built for the `astropy.nddata` object (although in many cases simply passing in an uncertainty array will also work):

In [ ]:
from astropy.nddata import StdDevUncertainty

unc = StdDevUncertainty(0.2 * np.ones(100))
spec1d_unc = Spectrum1D(spectral_axis=wavelength, flux=flux, uncertainty=unc)

# the errorbar matplotlib function does not natively support quantities
plt.errorbar(spec1d_unc.wavelength.value, 
             spec1d_unc.flux.value, 
             spec1d_unc.uncertainty.array,
             fmt='-',
             drawstyle='steps-mid',
             ecolor='k')

### Exercise

Create a `Spectrum1D` object for an ideal 5800 K blackbody and plot it. Try the same, but with (random) noise added and stored as the uncertainty.

Hint: while you can do this manually if you know the Planck function, there is an Astropy function to help you with this - you can find it via appropriate searches in the [astropy docs](http://docs.astropy.org).

## Loading spectra from files

Specutils also comes with readers for a variety of spectral data formats (including loaders for future JWST instruments). While support for specific formats depends primarily from users (like you!) providing readers, you may find that one has already been implemented for your favorite spectrum format.  As an example, we consider a spectrum form the [Sloan Digital Sky Survey](http://skyserver.sdss.org/):

In [ ]:
from urllib.request import urlretrieve

url = 'https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/1323/spec-1323-52797-0012.fits'
urlretrieve(url, 'data/sdss_spectrum.fits')

In [ ]:
sdss_spec = Spectrum1D.read('data/sdss_spectrum.fits', format='SDSS-III/IV spec')
plt.plot(sdss_spec.wavelength, sdss_spec.flux)

To see the full list of formats readable in your current version of specutils, see the table at the bottom of the `Spectrum1d.read` method:

In [ ]:
help(Spectrum1D.read)

### Exercise

If you have your own spectroscopic data, try loading a file here using either one of the built-in loaders, or the `Spectrum1D` interface, and plotting it.  If you don't have your own data on-hand, try downloading something of interest via a public archive (e.g., public HST data using MAST), and load it.

# Manipulating spectra

In addition to the analysis tools described in more detail in [the next notebook](Specutils_analysis.ipynb), Specutils also provides functionality for manipulating spectra.  In general these follow the pattern of creating *new* specutils objects with the results of the operation instead of in-place operations.

The most straightforward of operations are arithmetic manipulations.  In general these follow expected patterns. E.g.:

In [ ]:
newspec1d = spec1d - 2 * u.erg / u.s / (u.cm**2) / u.AA
plt.step(newspec1d.wavelength, newspec1d.flux)

However, when there is ambiguity in your intent - for example, two spectra with different units where it is not clear what the desired output is - errors are generally produced instead of the code attempting to guess:

In [ ]:
# This raises an error:
newspec1d - jyspec1d 

Resolving this requires explicit conversion:

In [ ]:
spec1d.new_flux_unit(u.Jy) - jyspec1d

### Exercise

Take the blackbody spectrum you generated above, and create a "spectral feature" by adding a Gaussian absorption or emission line to it using the arithmetic operators demonstrated above.  (Hint: [astropy.modeling](http://docs.astropy.org/en/stable/modeling/) contains [an implementation of the Gaussian line profile](http://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian1D.html#astropy.modeling.functional_models.Gaussian1D) which you may find useful.)

## Smoothing, etc.

More complex manipulation tools exist, outlined in the [relevant doc section](https://specutils.readthedocs.io/en/latest/manipulation.html). As a final example of this, we smooth our example spectrum using Gaussian smoothing:

In [ ]:
from specutils.manipulation import gaussian_smooth

smoothed_spec = gaussian_smooth(spec1d, 0.75)  # 0.75 pixel gaussian kernel
plt.step(smoothed_spec.wavelength, smoothed_spec.flux)

### Exercise

Try smoothing your spectrum loaded in the above example (or the SDSS spectrum).  Compare all available kernel types and decide which seems most appropriate for your spectrum.

# Next Steps

While the above focuses on loading and viewing spectra, the next notebook is aimed at quantitative scientific analysis of these spectra. Go to the [Specutils analysis notebook](Specutils_analysis.ipynb) for more on this.